In [ ]:
!curl ipecho.net/plain

34.19.69.43

In [ ]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 32.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from google.cloud import storage
import mysql.connector
import requests
import json
import os

# URL da chave pública na bucket
public_key_url = 'https://storage.googleapis.com/chaves-01/etl-cs/projeto-aline-440013-f24ad2833ee5.json'

# Fazer download da chave pública
response = requests.get(public_key_url)
response.raise_for_status()

# Salvar a chave em um arquivo temporário local
with open('service_account_key.json', 'w') as f:
    json.dump(response.json(), f)

# Configurar as credenciais para o Google Cloud Storage
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'service_account_key.json'

# Configuração do Google Cloud Storage e do banco de dados
bucket_name = 'banco-aline-01'
database_name = 'loja'
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)

# Conectar-se ao banco de dados MySQL
connection = mysql.connector.connect(
    host='34.69.233.123',
    user='root',
    password='',
    database=database_name
)

# Listar tabelas no banco de dados
cursor = connection.cursor()
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()

# Extraindo cada tabela e salvando na pasta do banco de dados na bucket
for (table_name,) in tables:
    # Carregar dados da tabela para um DataFrame
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, connection)

    # Nome do arquivo com o caminho da pasta
    csv_filename = f"{database_name}/{table_name}.csv"

    # Salvar o DataFrame em CSV temporário
    df.to_csv(table_name + '.csv', index=False)

    # Upload para a bucket GCS na pasta do banco
    blob = bucket.blob(csv_filename)
    blob.upload_from_filename(table_name + '.csv')
    print(f"Tabela {table_name} salva como {csv_filename} na bucket {bucket_name}.")

# Fechar o cursor e a conexão com o banco de dados
cursor.close()
connection.close()

<ipython-input-4-28cafc933aad>:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


Tabela Categoria salva como loja/Categoria.csv na bucket banco-aline-01.
Tabela Cliente salva como loja/Cliente.csv na bucket banco-aline-01.
Tabela ItemPedido salva como loja/ItemPedido.csv na bucket banco-aline-01.
Tabela Pedido salva como loja/Pedido.csv na bucket banco-aline-01.
Tabela Produto salva como loja/Produto.csv na bucket banco-aline-01.
Tabela ProdutoCategoria salva como loja/ProdutoCategoria.csv na bucket banco-aline-01.
